In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 158.4 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.3/484.3 kB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 128.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 100.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 48.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# === 모델 & 토크나이저 로드 ===
model_name = "Qwen/Qwen3-8B"
finetuned_dir = "./qwen3-lora-output/checkpoint-918"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)
model = PeftModel.from_pretrained(base_model, finetuned_dir)
model = model.to("cuda")
model.eval()

# === 간단한 시스템 프롬프트 설정 ===
system_prompt = """당신은 반려견 행동 문제를 상담해주는 전문가입니다.
사용자의 질문에 대해 반려견의 심리와 행동 원인을 분석하고,
이해를 돕는 설명과 함께 적절한 상담을 제공합니다."""

messages = [{"role": "system", "content": system_prompt}]

# === 대화 루프 ===
print("🐾 반려견 행동 상담 챗봇입니다. '완료'를 입력하면 종료됩니다.")

while True:
    user_input = input("\n🧑 사용자: ")
    if "완료" in user_input.strip():
        print("\n✅ 대화를 종료합니다.")
        break

    messages.append({"role": "user", "content": user_input})

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=2048,
            temperature=0.7,
            do_sample=True
        )

    output_ids = outputs[0][inputs.input_ids.shape[-1]:].tolist()

    try:
        end_token_id = 151668
        index = len(output_ids) - output_ids[::-1].index(end_token_id)
    except ValueError:
        index = 0

    thinking = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip()
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip()

    print(f"\n🧠 [thinking]: {thinking}")
    print(f"🤖 [assistant]: {content}")

    messages.append({"role": "assistant", "content": content})


tokenizer_config.json:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

...


🧠 [thinking]: <think>
</think>
🤖 [assistant]: 🔍 원인 분석:
안녕하세요. 황태규 훈련사입니다.🙂

💡 솔루션 제안:
다른 고민을 말씀해주시고, 이제부터 다른 고민을 알려주시기 바랍니다.



🧑 사용자:  갑자기 실내 배변을 안 하고 계속 밖에 나가서만 싸. 다시 실내 배변을 유도할 수 있을까?



🧠 [thinking]: <think>

</think>
🤖 [assistant]: 🔍 원인 분석:
안녕하세요 황태규 훈련사 입니다.🙂

💡 솔루션 제안:
실내 배변을 교육해보시면 좋습니다. 배변 패드를 이용하여 배변 교육을 진행해주세요. 배변 패드를 이용하여 교육을 진행해주시고, 배변패드에 배변을 잘 한 후에 보상을 해주세요. 배변 패드에 배변을 잘 한 후에도 실내 배변을 하지 않고 계속 실외에서 배변을 하게 된다면, 실내에서 배변을 하지 않는 이유가 배변패드를 이용하여 보상이 되지 않아 실내 배변을 하지 않을 수 있습니다. 실외 배변의 보상보다 실내 배변의 보상이 더 좋게 해주시면 좋겠습니다. 실내 배변을 하게 되면 실외 배변을 하는 것보다 보상이 더 좋아 보호자님께서 더욱 실내 배변을 유도할 수 있습니다.



🧑 사용자:  완료



✅ 대화를 종료합니다.
